En el pair programming de hoy debéis usar el csv que guardastéis cuando hicistéis el pairprgramming de codificicación (este csv debería tener las variables estadandarizas).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('once')

In [2]:
df = pd.read_pickle("datos/peliculas_limpio.pkl")
df.head(2)

,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Meta_score,No_of_Votes,Gross,Rating_BOX,Ryr_Sklstd,Rnt_Sklstd,Meta_Sklstd,Votes_Sklstd,Gross_Sklstd,Genre_map,Cert_map
0,1994.0,A,142,Drama,9.3,80.0,2343110.0,28341469.0,0.100485,-0.151515,0.676471,-0.001712,6.918768,0.131429,2,4.0
1,1972.0,A,175,Crime,9.2,100.0,1620367.0,134966411.0,0.100485,-0.818182,1.647059,1.426860,4.650520,1.997227,2,4.0


En pairprogramming anteriores ajustastéis vuestro datos a una regresión lineal. El objetivo de hoy es:
-Ajustar el modelo a un Decision Tree
-Extraer las métricas
-Debatid entre vosotras que modelo es mejor y por qué (basándose en las métricas)

In [3]:
# Separamos x e y
X = df.drop(["Released_Year", 'Genre','Certificate', 'Runtime','Meta_score','No_of_Votes','IMDB_Rating','Gross','Rating_BOX'], axis = 1)
y = df["Rating_BOX"]

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [6]:
lr = LinearRegression(n_jobs=-1)

In [7]:
lr.fit(x_train, y_train)

LinearRegression(n_jobs=-1)

In [8]:
y_predict_train = lr.predict(x_train) 
y_predict_test = lr.predict(x_test)

In [9]:
train_df = pd.DataFrame({'Real': y_train, 'Predicted': y_predict_train, 'Set': ['Train']*len(y_train)})
test_df  = pd.DataFrame({'Real': y_test,  'Predicted': y_predict_test,  'Set': ['Test']*len(y_test)})
resultados = pd.concat([train_df,test_df], axis = 0)
resultados.head()

,Real,Predicted,Set
29,0.100485,0.100485,Train
535,0.100485,0.100485,Train
695,0.100485,0.100485,Train
557,0.100485,0.100485,Train
836,0.100485,0.100485,Train


In [10]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df



In [11]:
arbol = DecisionTreeRegressor(random_state =0)

arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=0)

In [12]:
max_features = np.sqrt(len(x_train.columns))
max_features

2.6457513110645907

In [13]:
print(arbol.tree_.max_depth)

18


In [14]:
y_pred_test_dt = arbol.predict(x_test)
y_pred_train_dt = arbol.predict(x_train)

Hemos realizado una predicción basándonos en el decision tree y ahora comparamos los resultados entre test y train. Como vemos el MAE nos da resultados parecidos, MSE y RMSE también nos da resultados parecidos. Es en R2 donde vemos una clara diferencia, no se ajusta entre el train y el test. Vamos a probar con GridSearchCV para encontrar el mejor modelo de predicción.

In [15]:
dt_results1 = metricas(y_test, y_train, y_pred_test_dt, y_pred_train_dt, "Decission Tree I")
dt_results1

,MAE,MSE,RMSE,R2,set,modelo
0,2.785524e-11,1.104360e-21,3.323192e-11,0.053992,test,Decission Tree I
1,2.851399e-11,1.159559e-21,3.405230e-11,0.068495,train,Decission Tree I


In [16]:
# Hacemos el GridSearchCV

param = {"max_depth": [2,4,6],
        "max_features": [1,2,3,4],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]}

In [17]:
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(),
            param_grid= param,
            cv=10,
            return_train_score = True,
            scoring="neg_mean_squared_error")

In [18]:
gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 4, 6], 'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [19]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=6, max_features=2, min_samples_leaf=100,
                      min_samples_split=10)

In [20]:
y_pred_test_dt2 = mejor_modelo.predict(x_test)
y_pred_train_dt2 = mejor_modelo.predict(x_train)

In [21]:
dt_results2 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "Decision tree II")
dt_results2

,MAE,MSE,RMSE,R2,set,modelo
0,2.825680e-11,1.110035e-21,3.331718e-11,0.049131,test,Decision tree II
1,2.859512e-11,1.175846e-21,3.429062e-11,0.055411,train,Decision tree II


Con el GridSearch vemos que los resultados de R2 de test y train son muy parecidos lo que nos indica que hemos elegido bastante bien las variables predictoras.

In [22]:
# Juntamos los datos para comparar.
df_decision_results = pd.concat([dt_results1, dt_results2], axis = 0)
df_decision_results

,MAE,MSE,RMSE,R2,set,modelo
0,2.785524e-11,1.104360e-21,3.323192e-11,0.053992,test,Decission Tree I
1,2.851399e-11,1.159559e-21,3.405230e-11,0.068495,train,Decission Tree I
0,2.825680e-11,1.110035e-21,3.331718e-11,0.049131,test,Decision tree II
1,2.859512e-11,1.175846e-21,3.429062e-11,0.055411,train,Decision tree II


En los resultados del Decision Tree las métricas son buenas pero R2 nos da unos valores muy bajos con lo que apreciamos una mejoría en los resultados de Random Forest.